In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle, gzip
import copy
import math
from torchvision import datasets, transforms
from PIL import ImageOps
from torch.utils.data import ConcatDataset
import torch.utils.data as data_utils
from torch.autograd import Variable
from tqdm import tqdm
from IPython import display
from copy import deepcopy

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
code_size= 100

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Permuted MNIST

In [0]:
class PermutedMnistGenerator():
    def __init__(self, max_iter=10, random_seed=0):
        # Open data file
        f = gzip.open('/content/drive/My Drive/Jupyter/BayesML_Burnaev/CL_project/mnist.pkl.gz', 'rb')
        train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        f.close()

        # Define train and test data
        self.X_train = np.vstack((train_set[0], valid_set[0]))
        self.Y_train = np.hstack((train_set[1], valid_set[1]))
        self.X_test = test_set[0]
        self.Y_test = test_set[1]
        self.random_seed = random_seed
        self.max_iter = max_iter
        self.cur_iter = 0

        self.out_dim = 10           # Total number of unique classes
        self.class_list = range(10) # List of unique classes being considered, in the order they appear

        # self.classes is the classes (with correct indices for training/testing) of interest at each task_id
        self.classes = []
        for iter in range(self.max_iter):
            self.classes.append(range(0,10))

        self.sets = self.classes

    def get_dims(self):
        # Get data input and output dimensions
        return self.X_train.shape[1], self.out_dim

    def next_task(self):
        if self.cur_iter >= self.max_iter:
            raise Exception('Number of tasks exceeded!')
        else:
            np.random.seed(self.cur_iter+self.random_seed)
            perm_inds = list(range(self.X_train.shape[1]))
            # First task is (unpermuted) MNIST, subsequent tasks are random permutations of pixels
            if self.cur_iter > 0:
                np.random.shuffle(perm_inds)

            # Retrieve train data
            next_x_train = deepcopy(self.X_train)
            next_x_train = next_x_train[:,perm_inds]

            # Initialise next_y_train to zeros, then change relevant entries to ones, and then stack
            next_y_train = np.zeros((len(next_x_train), 10))
            next_y_train[:,0:10] = np.eye(10)[self.Y_train]

            # Retrieve test data
            next_x_test = deepcopy(self.X_test)
            next_x_test = next_x_test[:,perm_inds]

            next_y_test = np.zeros((len(next_x_test), 10))
            next_y_test[:,0:10] = np.eye(10)[self.Y_test]

            self.cur_iter += 1

            return next_x_train, next_y_train, next_x_test, next_y_test

    def reset(self):
        self.cur_iter = 0

# GAN

In [0]:
class GAN_generator(nn.Module):
    def __init__(self, code_size, output_size):
        super(GAN_generator, self).__init__()
        #n_features = 100
        #n_out = 784
        self.generator_net =  nn.Sequential(
            nn.Linear(code_size, 256),
            nn.LeakyReLU(0.2),            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, output_size)
        )

    def forward(self, x):
        return self.generator_net(x.cuda())
    
    
class GAN_discriminator(nn.Module):
    def __init__(self, input_size, output_size):
        super(GAN_discriminator, self).__init__()
        #n_features = 784
        #n_out = 1
        self.discriminator_net = nn.Sequential( 
            nn.Linear(input_size, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, output_size)
        )    

    def forward(self, x):
        return self.discriminator_net(x.cuda())

    
# Noise
def sample_noise_batch(batch_size, code_size):
    n = Variable(torch.randn(batch_size, code_size))
    return n.cuda() 
    
def images_to_vectors(images):
    return images.view(images.size(0), 784)

def vectors_to_images(vectors):
    return vectors.view(vectors.size(0), 1, 28, 28) 

def generator_loss(generator, discriminator, noise):
    generated_data = generator(noise)
    disc_on_generated_data = discriminator(generated_data)
    logp_gen_is_real = F.logsigmoid(disc_on_generated_data)
    loss = -torch.mean(logp_gen_is_real, 0)
    return loss

def discriminator_loss(discriminator, real_data, generated_data):
    disc_on_real_data = discriminator(real_data)
    disc_on_fake_data = discriminator(generated_data)
    logp_real_is_real = F.logsigmoid(disc_on_real_data)
    logp_gen_is_fake = F.logsigmoid(- disc_on_fake_data)
    loss = -torch.mean(logp_real_is_real, 0) - torch.mean(logp_gen_is_fake, 0)
    return loss

In [0]:
def sample_images(axs, generator, nrow, ncol, code_size, sharp=False):
    images = generator(sample_noise_batch(batch_size=nrow*ncol, code_size=code_size))
    images = images.data.cpu().numpy().reshape(nrow*ncol, 28, 28)
    if np.var(images)!=0:
        images = images.clip(0., 1.)#np.min(train_set[0]),np.max(train_set[0]))
    for i in range(nrow*ncol):
        ax = axs[i // ncol][i % ncol]
        if sharp:
            ax.imshow(images[i], cmap="gray", interpolation="none")
        else:
            ax.imshow(images[i], cmap="gray")
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        
    '''
        plt.subplot(nrow,ncol,i+1)
        if sharp:
            plt.imshow(images[i], interpolation="none")
        else:
            plt.imshow(images[i])
        
    plt.show()
    '''
    return 
def sample_probas(ax, discriminator, generator, batch_size, x_batch, code_size):
    plt.title('Generated vs real data')
    
    D_real = torch.sigmoid(discriminator(x_batch))
    generated_data_batch = generator(sample_noise_batch(batch_size, code_size))
    D_fake = torch.sigmoid(discriminator(generated_data_batch))
    
    ax.hist(D_real.data.cpu().numpy(),
             label='D(x)', alpha=0.5, range=[0,1])
    ax.hist(D_fake.data.cpu().numpy(),
             label='D(G(z))', alpha=0.5, range=[0,1])
    ax.legend(loc='best')
    #plt.show()


In [0]:
def GAN_train(discr_input, discr_output, gen_input, gen_output, batch_size, data_loader, n_epochs):
    d_losses = []
    g_losses = []
    max_g = 3
    axs = [[[] for _ in range(max_g)] for _ in range(2)]
    
    discriminator = GAN_discriminator(discr_input, discr_output)
    discriminator.cuda()
    generator = GAN_generator(gen_input, gen_output)
    generator.cuda()
    
    #optimizers
    disc_opt = torch.optim.Adam(discriminator.parameters(), lr=2e-4, betas=[0.5, 0.999])
    gen_opt = torch.optim.Adam(generator.parameters(), lr=2e-4, betas=[0.5, 0.999])
    
    for epoch in range(n_epochs):
        if  len(g_losses) == 0 or (g_losses[-1] < d_losses[-1] * 16): #Hack? Yeah, it's hack.
            d_loss = []
            for i in range(10):
                # Train discriminator
                data_loader_iter = iter(data_loader)
                x_batch, y_batch = next(data_loader_iter)
                fake_data = generator(sample_noise_batch(batch_size, gen_input)) #gen_input==code_size
                disc_loss = discriminator_loss(discriminator, x_batch, fake_data)
                disc_opt.zero_grad()
                disc_loss.backward()
                disc_opt.step()
                d_loss.append(disc_loss.data.cpu().numpy()[0])

                
        if len(d_losses) > 0:
            d_losses.append(np.mean(d_loss) * 0.05 + d_losses[-1] * (1 - 0.05))
        else:
            d_losses.append(np.mean(d_loss))

        # Train generator
        noise = sample_noise_batch(batch_size, gen_input)
        gen_loss = generator_loss(generator, discriminator, noise)
        gen_opt.zero_grad()
        gen_loss.backward()
        gen_opt.step()
        
        if len(g_losses) > 0:
            g_losses.append(gen_loss.data.cpu().numpy()[0] * 0.05 + g_losses[-1] * (1 - 0.05))
        else:
            g_losses.append(gen_loss.data.cpu().numpy()[0])
    
        
        #plot results
        if epoch %100==0:
           
            #display.clear_output(True)
            '''
            plt.figure(figsize=(18, 4))

            ax = plt.subplot2grid((2,3 + max_g + 2), (0,0), colspan=3, rowspan=2)
            ax.set_title("D-Loss")
            ax.set_xlabel("#iteration")
            ax.set_ylabel("loss")
            ax.plot(g_losses, 'r', label='G-loss', alpha=0.8)
            ax.plot(d_losses, 'b', label='D-loss', alpha=0.8)
            ax.legend()
            ax.grid(True)

            for i in range(0, 2):
                for j in range(3, 3 + max_g):
                    axs[i][j - 3] = plt.subplot2grid((2, 3 + max_g + 2), (i, j))
            sample_images(axs, generator, 2, max_g, gen_input, True)

            ax = plt.subplot2grid((2, 3 + max_g + 2), (0, 3 + max_g), colspan=2, rowspan=2)
            prev_hist = sample_probas(ax, discriminator, generator, 1000, x_batch, gen_input)
            plt.tight_layout()
            plt.show()

            
            sample_images(generator, 2, 3, gen_input, sharp=True)
            sample_probas(discriminator, generator, 1000, x_batch, gen_input)
            '''
            print('epoch {}\n'.format(epoch))
            print('dlosses {}\n'.format(d_losses))
            print('glosses {}\n'.format(g_losses))

    return generator

# Solver

In [0]:
class BayesLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        # Weight parameters 1-dim params for each layer
        self.weight_mu = nn.Parameter(torch.zeros(out_features, in_features))
        self.weight_sigma = nn.Parameter(1e-6 * torch.ones(out_features, in_features))
        self.weight = torch.distributions.Normal(self.weight_mu, self.weight_sigma)
        
        # Prior distribution
        self.weight_prior_mu = torch.Tensor([0.]).to(device) #torch.zeros(in_features)
        self.weight_prior_sigma = torch.Tensor([1.]).to(device) #torch.ones(in_features)
        #self.weight_prior = torch.distributions.Normal(self.weight_prior_mu, self.weight_prior_sigma)
        
        self.log_prior = 0.
        self.log_variational_posterior = 0.

    def forward(self, input, sampling=False, calculate_log_probs=False):
        if self.training or sampling:
            weight = self.weight.sample()
        else:
            weight = self.weight.mean
        '''
        if self.training or calculate_log_probs:
            self.log_prior = self.weight_prior.log_prob(weight)
            self.log_variational_posterior = self.weight.log_prob(weight)
        else:
            self.log_prior, self.log_variational_posterior = 0., 0.
        '''
        return F.linear(input, weight)
    
    def kl(self):
        return (torch.log(torch.sqrt(self.weight_prior_sigma)) - torch.log(torch.sqrt(self.weight_sigma)) + (
            (self.weight_mu - self.weight_prior_mu) ** 2 + self.weight_sigma - self.weight_prior_sigma) / (
            2. * self.weight_prior_sigma)).sum()
    

In [0]:
class Solver(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        #hidden_size=100
        super(Solver, self).__init__()
        self.l1 = BayesLinear(input_size, hidden_size)
        self.l2 = BayesLinear(hidden_size, hidden_size)
        self.l3 = BayesLinear(hidden_size, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x, sampling=False):
        output = self.l1(x, sampling=sampling)
        output = self.relu(output)
        output = self.l2(output, sampling=sampling)
        output = self.relu(output)
        return self.l3(output, sampling=sampling)
    
    
    def log_prior(self):
        return self.l1.log_prior \
               + self.l2.log_prior \
               + self.l3.log_prior
    
    def log_variational_posterior(self):
        return self.l1.log_variational_posterior \
               + self.l2.log_variational_posterior \
               + self.l2.log_variational_posterior
    
    
    def loss(self, gans, solvers, x_curr, y_curr):
        kl = self.l1.kl() + self.l2.kl() + self.l3.kl()
        sumloglik = 0.
        if len(gans) > 0:
            for gan, solver in zip(gans[:-1], solvers[:-1]):
                x_gan = gan(sample_noise_batch(batch_size=len(x_curr), code_size=code_size))
                y_gan = solver(x_gan, sampling=False)
                logits = self.forward(x_gan, sampling=False)
                sumloglik += F.binary_cross_entropy_with_logits(logits, y_gan)
            x_gan_new = gans[-1](sample_noise_batch(batch_size=60000, code_size=code_size))
            y_gan_new = solvers[-1](x_gan_new, sampling=False)
            logits = self.forward(torch.cat([x_gan_new, x_curr],dim=0), sampling=False)
            sumloglik += F.binary_cross_entropy_with_logits(logits, torch.cat([y_gan_new, y_curr],dim=0)) 
                
        else:
            logits = self.forward(x_curr, sampling=False)
            sumloglik = F.binary_cross_entropy_with_logits(logits, y_curr)
            
        return kl + sumloglik #actually negsumloglik

In [0]:
def solver_train_predict(input_size, hidden_size, output_size, batch_size, train_loader, n_epochs, seen_tasks,
                         gans, solvers):
    net = Solver(input_size, hidden_size, output_size)
    net.to(device)
    optimizer = optim.Adam(net.parameters())
    
    #training
    net.train()
    loss_list = []
    for epoch in tqdm(range(n_epochs)):
        cumulative_loss = 0.
    
        for x_train, y_train in train_loader:
            x_train = x_train.to(device)
            y_train = y_train.to(device)
            
            net.zero_grad()
            loss = net.loss(gans, solvers, x_train, y_train)
            loss.backward()
            optimizer.step()
            
            cumulative_loss += loss.detach().cpu().numpy()
       
        loss_list.append(cumulative_loss)
        print('solver_losses = {}\n'.format(loss_list))
    '''
    #prediction
    preds = []
    net.eval()
    with torch.no_grad():
        for x_test, y_test in test_loader:
            x_test = x_test.to(device)
            y_test = y_test.to(device)
            logits = net(x_test, sampling=True)
            pred_idx = logits.max(1, keepdims=True)[1].transpose(0,1).detach().cpu().numpy()[0]
            y_pred = np.zeros(y_test.size())
            y_pred[:,0:10] = torch.Tensor(np.eye(10)[pred_idx])
        print('pred = {}, test = {}'.format(y_pred,y_test))
    '''
    return net

# Main loop

In [0]:
def scholar_train(n_tasks=5, batch_size=256, gan_epochs=301, solver_epochs=5):
    gans = []
    solvers = []
    data_gen = PermutedMnistGenerator(max_iter=n_tasks, random_seed=0)
    print('\n dataset generated, starting tasks')
    for task in tqdm(range(n_tasks)):
        #load task data
        x_new_train, y_new_train, x_new_test, y_new_test = data_gen.next_task()
        print('task {} data loaded'.format(task))
        if task !=0:
            '''
            #cocnatenate with prev GAN data
            x_train = torch.cat([torch.Tensor(x_new_train), x_gen], 0).to(device)
            y_train = torch.cat([torch.Tensor(y_new_train), y_gen_pred], 0).to(device) 
            
            train = data_utils.TensorDataset(x_train, y_train)
            train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)
            '''
            #train curr task Solver
            print('running solver task {}'.format(task))
            curr_solver = solver_train_predict(input_size=784, hidden_size=100, output_size=10, batch_size=batch_size, 
                             train_loader=train_loader, n_epochs=solver_epochs, seen_tasks=task, gans=gans, solvers=solvers)
            solvers.append(curr_solver)
            
            #train curr task GAN
            print('running GAN task {}'.format(task))
            curr_generator = GAN_train(discr_input=784, discr_output=1, gen_input=code_size, gen_output=784, 
                                   batch_size=batch_size, data_loader=train_loader, n_epochs=gan_epochs)
            gans.append(curr_generator)
            
            
        else:
            print('running task {}'.format(task))
            x_train = torch.Tensor(x_new_train).to(device)
            y_train = torch.Tensor(y_new_train).to(device)
            train = data_utils.TensorDataset(x_train, y_train)
            train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

            #train first Solver
            print('running solver task {}'.format(task))
            curr_solver = solver_train_predict(input_size=784, hidden_size=100, output_size=10, batch_size=batch_size, 
                             train_loader=train_loader, n_epochs=solver_epochs, seen_tasks=task, gans=gans, solvers=solvers)
            solvers.append(curr_solver)
            
            #train first GAN
            print('running GAN task {}'.format(task))
            curr_generator = GAN_train(discr_input=784, discr_output=1, gen_input=code_size, gen_output=784, 
                                   batch_size=batch_size, data_loader=train_loader, n_epochs=gan_epochs)
            gans.append(curr_generator)
        print('gans after task {}: {}'.format(task, gans))
        print('solvers after task {}: {}'.format(task, solvers))
        
        ''' 
        x_gen = gan_generators[0](sample_noise_batch(batch_size=6000, code_size=code_size))
        if len(gan_generators) > 1:
            for i in range(1, len(gan_generators)):
                curr_x_gen =  gan_generators[i](sample_noise_batch(batch_size=6000, code_size=code_size))           
                x_gen = torch.cat([x_gen, curr_x_gen], 0)
        #train solver
        y_gen_pred = solver_train_predict(#x_gen...)
        '''
        

In [41]:
scholar_train(n_tasks=2, batch_size=256, gan_epochs=301, solver_epochs=200)


  0%|          | 0/2 [00:00<?, ?it/s]


 dataset generated, starting tasks
task 0 data loaded
running task 0




  0%|          | 0/5 [00:00<?, ?it/s]

running solver task 0




 20%|██        | 1/5 [00:01<00:05,  1.50s/it]

...

solver_losses = [43881169.875, 42063100.75, 40899046.5, 39685969.234375]





100%|██████████| 5/5 [07:52<00:00, 94.48s/it]



solver_losses = [43881169.875, 42063100.75, 40899046.5, 39685969.234375, 38497550.609375]

running GAN task 1
epoch 0

dlosses [1.1277566]

glosses [0.79874367]

epoch 100

dlosses [1.1277566, 1.0990544736385346, 1.0556731632351877, 1.0083886109739544, 0.9631030070833863, 0.9191359882702306, 0.8757713441940527, 0.8336708319333952, 0.7932278417153034, 0.7544894801736105, 0.717388494032345, 0.6819395334403571, 0.6481406121461281, 0.6159681260954095, 0.5853667614306972, 0.556267888095408, 0.5285869388900527, 0.5022557897829103, 0.47721963418345853, 0.453422160297792, 0.4308079320349851, 0.4093202472897873, 0.38891048840951536, 0.36950912812043507, 0.35107043489888434, 0.3335483703608729, 0.31693213339716725, 0.30114670828164686, 0.2861505544219025, 0.2719042082551454, 0.2583701793967261, 0.24551285198122771, 0.23329839093650429, 0.22169465294401702, 0.21067110185115412, 0.20019872831293437, 0.1902499734516256, 0.18079865633338227, 0.1718199050710511, 0.1632900913718365, 0.1551867683575826


100%|██████████| 2/2 [08:34<00:00, 164.02s/it]


epoch 300

dlosses [1.1277566, 1.0990544736385346, 1.0556731632351877, 1.0083886109739544, 0.9631030070833863, 0.9191359882702306, 0.8757713441940527, 0.8336708319333952, 0.7932278417153034, 0.7544894801736105, 0.717388494032345, 0.6819395334403571, 0.6481406121461281, 0.6159681260954095, 0.5853667614306972, 0.556267888095408, 0.5285869388900527, 0.5022557897829103, 0.47721963418345853, 0.453422160297792, 0.4308079320349851, 0.4093202472897873, 0.38891048840951536, 0.36950912812043507, 0.35107043489888434, 0.3335483703608729, 0.31693213339716725, 0.30114670828164686, 0.2861505544219025, 0.2719042082551454, 0.2583701793967261, 0.24551285198122771, 0.23329839093650429, 0.22169465294401702, 0.21067110185115412, 0.20019872831293437, 0.1902499734516256, 0.18079865633338227, 0.1718199050710511, 0.1632900913718365, 0.15518676835758263, 0.14748861149404147, 0.14017536247367735, 0.13322777590433144, 0.12662756866345282, 0.12035737178461813, 0.11440068474972517, 0.10874183206657687, 0.1033659220